In [ ]:
import numpy as np

dataset_path = "datasets/dataset_light.npy"
loaded_dataset = np.load(dataset_path, allow_pickle=True).item()

print("Loaded dataset keys:")
for key in loaded_dataset.keys():
    print(f"- {key}")

'''print("\nLoaded dataset values:")
for key, value in loaded_dataset.items():
    print(f"- {key}: {value}")'''

# put gain inside a new dictionnary with a key named values
new_dataset = loaded_dataset.copy()
new_dataset["values"] = {"gain": loaded_dataset["gain"]}

# add a new key named name with values from 1 to length of gain
new_dataset["name"] = [str(i) for i in range(len(loaded_dataset["gain"]))]

print("\nNew dataset keys:")
for key in new_dataset.keys():
    print(f"- {key}")

print("\nLength of new dataset data:")
print(f"- {len(new_dataset['data'])}")

# save the new dataset to a file
new_dataset_path = "datasets/dataset_light_correct.npy"
np.save(new_dataset_path, new_dataset, allow_pickle=True)

'''# show the profiles of the new dataset
import matplotlib.pyplot as plt

profiles = new_dataset["data"]
plt.figure(figsize=(10, 6))
for i in range(len(profiles)):
    plt.plot(profiles[i], label=f"Profile {i+1}")
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Profiles of the New Dataset")
plt.legend()
plt.grid()
plt.show()'''

In [ ]:
# check all itmes in the dataset
import numpy as np

dataset_path = "datasets/dataset800.npy"
loaded_dataset = np.load(dataset_path, allow_pickle=True).item()

print("Loaded dataset keys:")
for key in loaded_dataset.keys():
    print(f"- {key}")

print("\nLoaded dataset values:")
for key in loaded_dataset["values"].keys():
    print(f"- {key}")

print("\nLoaded dataset coil pitch:")
print(f"- {loaded_dataset['data'][0]}")

print("\nLoaded dataset time:")
print(f"- {loaded_dataset['time']}")

print("Number of profiles:")
print(f"- {len(loaded_dataset['data'])}")

Loaded dataset keys:
- values
- data
- time
- name

Loaded dataset values:
- cutoff
- qsup
- e95
- e99
- coil_E
- Mgain
- gain

Loaded dataset coil pitch:
- [0.00084004 0.0008322  0.00082359 0.00081411 0.00080368 0.00079219
 0.00077958 0.00076577 0.00075074 0.00073449 0.0007171  0.00069873
 0.0006796  0.00066002 0.00064034 0.0006209  0.00060206 0.00058408
 0.00056716 0.00055142 0.0005369  0.0005236  0.00051147 0.00050044
 0.00049042 0.00048131 0.00047304 0.00046551 0.00045865 0.00045238
 0.00044665 0.00044138 0.00043655 0.00043209 0.00042797 0.00042416
 0.00042063 0.00041734 0.00041428 0.00041142]

Loaded dataset time:
- [0.         0.00051282 0.00102564 0.00153846 0.00205128 0.0025641
 0.00307692 0.00358974 0.00410256 0.00461538 0.00512821 0.00564103
 0.00615385 0.00666667 0.00717949 0.00769231 0.00820513 0.00871795
 0.00923077 0.00974359 0.01025641 0.01076923 0.01128205 0.01179487
 0.01230769 0.01282051 0.01333333 0.01384615 0.01435897 0.01487179
 0.01538462 0.01589744 0.01641026 0.0